Bo Yang (Aaron) USCID:7526922531

### INF 552 HOMEWORK_7  Date: 12-01-2019

# 1. Generative Models for Text

## (a) In this problem, we are trying to build a generative model to mimic the writing style of prominent British Mathematician, Philosopher, prolific writer, and political activist, Bertrand Russell.

## (b) Download the following books from Project Gutenberg http://www.gutenberg.org/ebooks/author/355 in text format:
- i. The Problems of Philosophy
- ii. The Analysis of Mind
- iii. Mysticism and Logic and Other Essays
- iv. Our Knowledge of the External World as a Field for Scientific Method in Philosoph

Project Gutenberg adds a standard header and footer to each book and this is not part of the original text. Open the file in a text editor and delete the header and footer.

The header is obvious and ends with the text:<br>
\*** <b>START OF THIS PROJECT GUTENBERG EBOOK AN INQUIRY INTO MEANING AND TRUTH</b> \***<br>
The footer is all of the text after the line of text that says:<br>
<b>THE END</b>

To have a better model, it is strongly recommended that you download the following books from The Library of Congress https://archive.org and convert them to text files:
- i. The History of Western Philosophy https://archive.org/details/westernphilosophy4
- ii. The Analysis of Matter https://archive.org/details/in.ernet.dli.2015.221533
- iii. An Inquiry into Meaning and Truth https://archive.org/details/BertrandRussell-AnInquaryIntoMeaningAndTruth
<br>

Try to only use the text of the books and throw away unwanted text before and after the text, although in a large corpus, these are considered as noise and should not make big problems.

## (c) LSTM: Train an LSTM to mimic Russell’s style and thoughts:

### i. Concatenate your text files to create a corpus of Russell’s writings

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
import tensorflow as tf
from tensorflow import keras
import sys
import copy

for module in sklearn, tf, keras:
    print("{}'s version is {}".format(module.__name__, module.__version__))

sklearn's version is 0.21.3
tensorflow's version is 2.0.0-alpha0
tensorflow.python.keras.api._v2.keras's version is 2.2.4-tf


In [3]:
ROOT_PATH = "./assets"
SPLASH = "/"
PART_DATA_FILES = "partbks/"
ALL_DATA_FILES = "books/"

In [4]:
import string
import os

def gen_corpus(file_pathes, output_file_name="CORPIS.txt"):
    dataFiles = os.listdir(file_pathes)
    resultFile = open(output_file_name, 'w')
    for fileItem in dataFiles:
        with open(file_pathes + SPLASH + fileItem, 'r', 
                  encoding='ascii', errors='ignore') as fileDataStream:
            for line in fileDataStream:
                resultFile.write(line)

    resultFile.close()
    return resultFile.name


def gen_chars_set(orginalText, ignore_case=False, remove_punctuation=False):
    trimedText = copy.copy(orginalText)
    charSet = set(orginalText)
    if ignore_case:
        trimedText = orginalText.lower()
        charSet = set(orginalText.lower())
    if remove_punctuation:
        charSet = charSet.difference(set(string.punctuation))
        trimedText = trimedText.translate(str.maketrans('', '', string.punctuation))
    return trimedText, charSet

In [5]:
resultFile = gen_corpus(ROOT_PATH + SPLASH + ALL_DATA_FILES)
contents = open(resultFile, 'r').read()
print("The length of corpus of Russell's writing is: ", len(contents))

The length of corpus of Russell's writing is:  5095252


### ii. Use a character-level representation for this model by using extended ASCII that has N = 256 characters. Each character will be encoded into a an integer using its ASCII code. Rescale the integers to the range [0, 1], because LSTM uses a sigmoid activation function. LSTM will receive the rescaled integers as its input.

In [6]:
from sklearn.preprocessing import MinMaxScaler
import operator

trimmedContent, CharacterSet = gen_chars_set(contents, ignore_case=True, remove_punctuation=True)
char2AsciiDict = dict()
for index, char in enumerate(sorted(CharacterSet)):
    char2AsciiDict[char] = index

print("--------------The original dict of distinct characters with its ascii is: 
      --------------\n", sorted(char2AsciiDict.items(), key=operator.itemgetter(1)))
scaledCharDict = dict()
scaledValues = MinMaxScaler().fit_transform(np.array(list(char2AsciiDict.values())).reshape(-1, 1))
for index in range(len(scaledValues)):
    scaledCharDict[list(char2AsciiDict.keys())[index]] = scaledValues[index][0]

print("--------------The scaled dict of distinct characters with its ascii is: 
      --------------\n", sorted(scaledCharDict.items(), key=operator.itemgetter(1)))

ascii2CharDict = {v:k for k, v in char2AsciiDict.items()}
print("--------------The reversed dict of distinct characters with its ascii is: 
      --------------\n", sorted(ascii2CharDict.items(), key=operator.itemgetter(1)))

--------------The original dict of distinct characters with its ascii is: --------------
 [('\n', 0), (' ', 1), ('0', 2), ('1', 3), ('2', 4), ('3', 5), ('4', 6), ('5', 7), ('6', 8), ('7', 9), ('8', 10), ('9', 11), ('a', 12), ('b', 13), ('c', 14), ('d', 15), ('e', 16), ('f', 17), ('g', 18), ('h', 19), ('i', 20), ('j', 21), ('k', 22), ('l', 23), ('m', 24), ('n', 25), ('o', 26), ('p', 27), ('q', 28), ('r', 29), ('s', 30), ('t', 31), ('u', 32), ('v', 33), ('w', 34), ('x', 35), ('y', 36), ('z', 37)]
--------------The scaled dict of distinct characters with its ascii is: --------------
 [('\n', 0.0), (' ', 0.02702702702702703), ('0', 0.05405405405405406), ('1', 0.08108108108108109), ('2', 0.10810810810810811), ('3', 0.13513513513513514), ('4', 0.16216216216216217), ('5', 0.1891891891891892), ('6', 0.21621621621621623), ('7', 0.24324324324324326), ('8', 0.2702702702702703), ('9', 0.2972972972972973), ('a', 0.32432432432432434), ('b', 0.35135135135135137), ('c', 0.3783783783783784), ('d', 0.40

### iii. Choose a window size, e.g., W = 100
### iv. Inputs to the network will be the first W −1 = 99 characters of each sequence, and the output of the network will be the Wth character of the sequence. Basically, we are training the network to predict each character using the 99 characters that precede it. Slide the window in strides of S = 1 on the text. For example, if W = 5 and S = 1 and we want to train the network with the sequence ABRACADABRA, The first input to the network will be ABRA and the corresponding output will be C. The second input will be BRAC and the second output will be A, etc.

In [7]:
windowSize = 99
X_data = []
Y_char_data = []
for i in range(0, len(trimmedContent) - windowSize, 1):
    tempBlock = trimmedContent[i:i + windowSize]
    theLastChar = trimmedContent[i + windowSize]
    X_data.append([scaledCharDict[char] for char in tempBlock])
    Y_char_data.append(char2AsciiDict[theLastChar])

### v. Note that the output has to be encoded using a one-hot encoding scheme with N = 256 (or less) elements. This means that the network reads integers, but outputs a vector of N = 256 (or less) elements.

In [8]:
numBlocks = len(X_data)
X_data = np.reshape(X_data, (numBlocks, windowSize, 1))
Y_data = keras.utils.to_categorical(Y_char_data)
print(X_data.shape)
print(Y_data.shape)

(4942256, 99, 1)
(4942256, 38)


### vi. Use a single hidden layer for the LSTM with N = 256 (or less) memory units.

### vii. Use a Softmax output layer to yield a probability prediction for each of the characters between 0 and 1. This is actually a character classification problem with N classes. Choose log loss (cross entropy) as the objective function for the network (research what it means).

### viii. We do not use a test dataset. We are using the whole training dataset to learn the probability of each character in a sequence. We are not seeking for a very accurate model. Instead we are interested in a generalization of the dataset that can mimic the gist of the text.

In [14]:
memory_units = 256

model = keras.models.Sequential([
    keras.layers.LSTM(units=memory_units,
                      input_shape=(X_data.shape[1], X_data.shape[2])),
    keras.layers.Dense(Y_data.shape[1], activation="softmax"),
])
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm (UnifiedLSTM)   (None, 256)               264192    
_________________________________________________________________
dense (Dense)                (None, 38)                9766      
Total params: 273,958
Trainable params: 273,958
Non-trainable params: 0
_________________________________________________________________


### ix. Choose a reasonable number of epochs for training, considering your computational power (e.g., 30, although the network will need more epochs to yield a better model).

In [15]:
epochs = 30
output_dir = "./text_generation_checkpoint1"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
checkpoint_prefix = os.path.join(output_dir, 'ck_{epoch:02d}.hdf5')
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix,
    monitor='loss',
    save_weights_only= True,
    mode='min'
)
earlyStoppingCk = keras.callbacks.EarlyStopping(patience = 5, min_delta = 1e-4)
history = model.fit(X_data, Y_data, epochs=epochs, batch_size=128, 
                    callbacks=[checkpoint_callback, earlyStoppingCk])

Epoch 1/30
4942256/4942256 [==============================] - 444s 90us/sample - loss: 2.6819
Epoch 2/30
4942256/4942256 [==============================] - 439s 89us/sample - loss: 2.5241
Epoch 3/30
4942256/4942256 [==============================] - 439s 89us/sample - loss: 2.4181
Epoch 4/30
4942256/4942256 [==============================] - 440s 89us/sample - loss: 2.3259
Epoch 5/30
4942256/4942256 [==============================] - 439s 89us/sample - loss: 2.2497
Epoch 6/30
4942256/4942256 [==============================] - 440s 89us/sample - loss: 2.1865
Epoch 7/30
4942256/4942256 [==============================] - 438s 89us/sample - loss: 2.1323
Epoch 8/30
4942256/4942256 [==============================] - 438s 89us/sample - loss: 2.0826
Epoch 9/30
4942256/4942256 [==============================] - 438s 89us/sample - loss: 2.0390
Epoch 10/30
4942256/4942256 [==============================] - 438s 89us/sample - loss: 2.0018
Epoch 11/30
4942256/4942256 [==============================

### x. Use model checkpointing to keep the network weights to determine each time an improvement in loss is observed at the end of the epoch. Find the best set of weights in terms of loss.

In [17]:
minLossCheckPoint = output_dir +'/ck_30.hdf5'
model.load_weights(minLossCheckPoint)
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4))

### xi. Use the network with the best weights to generate 1000 characters, using the following text as initialization of the network:
    "There are those who take mental phenomena naively, just as they
    would physical phenomena. This school of psychologists tends not to
    emphasize the object."

In [19]:
initText = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'
initText = initText.translate(str.maketrans('', '', string.punctuation))

In [26]:
genText = copy.copy(initText.lower())
encodeList = [scaledCharDict[char] for char in genText][-99:]
for _ in range(1000):
    data = np.reshape(encodeList, (1, len(encodeList), 1))
    pred = model.predict(data)
    charIndex = np.argmax(pred)
    char = ascii2CharDict[np.argmax(pred)]
    genText += char
    encodeList.append(scaledCharDict[char])
    encodeList = encodeList[1:len(encodeList)]
    
print(genText)

there are those who take mental phenomena naively just as they would physical phenomena this school of psychologists tends not to emphasize the object much exposure to content there  to be any way to control of the semse and the semse of the 
perception of the semse in the sense of the soace and the 
content of the soace and the soace and the soace and the soace of the content of the soace 
of the soace and the soace and the soace and the soace and the 



170  























































































































































































































































































































































































































































































































































### xii. Extra Practice: Use one-hot encoding for the input sequence. Use a large number of epochs, e.g., 150. Add dropout to the network, and use a deeper LSTM (e.g. with 3 or more layers). Generate 3000 characters using the above initialization and report if you get more meaningful text.

In [10]:
model2 = keras.models.Sequential([
    keras.layers.LSTM(units=memory_units,
                      input_shape=(X_data.shape[1], X_data.shape[2]), 
                      return_sequences=True),
    
    keras.layers.LSTM(units=memory_units, return_sequences=True),
    keras.layers.LSTM(units=memory_units, return_sequences=False),
    keras.layers.Dense(Y_data.shape[1], activation="softmax"),
])
model2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4))
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
unified_lstm (UnifiedLSTM)   (None, 99, 256)           264192    
_________________________________________________________________
unified_lstm_1 (UnifiedLSTM) (None, 99, 256)           525312    
_________________________________________________________________
unified_lstm_2 (UnifiedLSTM) (None, 256)               525312    
_________________________________________________________________
dense (Dense)                (None, 38)                9766      
Total params: 1,324,582
Trainable params: 1,324,582
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs = 100
output_dir = "./text_generation_checkpoint2"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
    
checkpoint_prefix2 = os.path.join(output_dir, 'ck_{epoch:02d}.hdf5')
checkpoint_callback2 = keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_prefix2,
    monitor='loss',
    save_weights_only= True,
    mode='min'
)

history2 = model2.fit(X_data, Y_data, epochs=epochs, batch_size=4096, 
                      callbacks=[checkpoint_callback2, earlyStoppingCk])

Epoch 1/100
4942256/4942256 [==============================] - 696s 141us/sample - loss: 2.5840
Epoch 2/100
4942256/4942256 [==============================] - 696s 141us/sample - loss: 2.4899
Epoch 3/100
4942256/4942256 [==============================] - 696s 141us/sample - loss: 2.3660
Epoch 4/100
4942256/4942256 [==============================] - 696s 141us/sample - loss: 2.2418
Epoch 5/100
4942256/4942256 [==============================] - 695s 141us/sample - loss: 2.1290
Epoch 6/100
4942256/4942256 [==============================] - 695s 141us/sample - loss: 2.0347
Epoch 7/100
4942256/4942256 [==============================] - 695s 141us/sample - loss: 1.9565
Epoch 8/100
4942256/4942256 [==============================] - 696s 141us/sample - loss: 1.8915
Epoch 9/100
4942256/4942256 [==============================] - 696s 141us/sample - loss: 1.8359
Epoch 10/100
4942256/4942256 [==============================] - 695s 141us/sample - loss: 1.7877
Epoch 11/100
4942256/4942256 [=========

In [11]:
minLossCheckPoint = output_dir +'/ck_33.hdf5'
model2.load_weights(minLossCheckPoint)
model2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(1e-4))

In [20]:
genText2 = copy.copy(initText.lower())
encodeList2 = [scaledCharDict[char] for char in genText2][-99:]
for _ in range(3000):
    data2 = np.reshape(encodeList2, (1, len(encodeList2), 1))
    pred2 = model2.predict(data2)
    charIndex2 = np.argmax(pred2)
    char2 = ascii2CharDict[charIndex2]
    genText2 += char2
    encodeList2.append(scaledCharDict[char2])
    encodeList2 = encodeList2[1:len(encodeList2)]
    
print(genText2)

there are those who take mental phenomena naively just as they would physical phenomena this school of psychologists tends not to emphasize the object much exposure to content there to be any way to control of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense of the sense 

### xiii. Extra Practice- HMM: Train a Hidden Markov Model with V hidden states and V possible outputs using Baum-Welch Algorithm (or any other modern algorithm that is available) with the Russell corpus, where V is the number of distinct words in the corpus. Note that for HMM, you NOT use character level encoding, because it may yield totally meaningless results, although the transition matrices associated with it will be way smaller (you are welcome to try it). Generate 200 words using the model and comment on its meaningfulness. Extra extra practice: can you train a higher order HMM (i.e. an HMM that assumes dependency on more than one previous state) to get a better model?